In [1]:
import os
import jams
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from joblib import Parallel, delayed
import mir_eval
import pumpp

In [2]:
def getid(fname):
    return os.path.splitext(os.path.basename(fname))[0]

In [3]:
def chord_to_hier(ann):
        
    C3 = pumpp.task.ChordTagTransformer(vocab='3')
    C35 = pumpp.task.ChordTagTransformer(vocab='35')
    C3567s = pumpp.task.ChordTagTransformer(vocab='3567s')
    
    ann_h = jams.Annotation('multi_segment',
                            time=ann.time,
                            duration=ann.duration,
                            annotation_metadata=ann.annotation_metadata)
    
    for obs in ann:
        root = C3.simplify(obs.value)
        if ':' in root:
            root = root[:root.index(':')]
        ann_h.append(time=obs.time, duration=obs.duration,
                    value=dict(level=1, label=root),
                    confidence=obs.confidence)
        
        ann_h.append(time=obs.time, duration=obs.duration,
                    value=dict(level=2, label=C3.simplify(obs.value)),
                    confidence=obs.confidence)
        ann_h.append(time=obs.time, duration=obs.duration,
                    value=dict(level=3, label=C35.simplify(obs.value)),
                    confidence=obs.confidence)
        ann_h.append(time=obs.time, duration=obs.duration,
                    value=dict(level=4, label=C3567s.simplify(obs.value)),
                    confidence=obs.confidence)
        ann_h.append(time=obs.time, duration=obs.duration,
                    value=dict(level=5, label=obs.value),
                    confidence=obs.confidence)
    return ann_h

In [39]:
def drop_empty(ann):
    
    for obs in ann.pop_data():
        if obs.duration:
            ann.append(time=obs.time, duration=obs.duration, value=obs.value, confidence=obs.confidence)

In [40]:
def compare(ref_file, est_file, corpus):
    
    ref_jam = jams.load(ref_file, validate=False)
    est_jam = jams.load(est_file, validate=False)
    
    ref_ann = ref_jam.search(corpus=corpus)[0]
    drop_empty(ref_ann)
    
    est_ann = est_jam.annotations[0]
    drop_empty(est_ann)
    
    results = jams.eval.hierarchy(chord_to_hier(ref_ann), chord_to_hier(est_ann))
    
    return {getid(est_file): dict(results)}

In [41]:
from joblib import Parallel, delayed

In [42]:
def score_track(tid, corpus, path):
    try:
        ref_file = os.path.join(REF_PATH, os.path.extsep.join([tid, 'jams']))
        est_file = os.path.join(path, os.path.extsep.join([tid, 'jams']))
        return compare(ref_file, est_file, corpus)
    except ValueError as exc:
        print('Failure at track {}'.format(est_file))
        print(exc)
    return dict()

def eval_model(ids, corpus, path, n_jobs=18):
    
    results_arr = Parallel(n_jobs=n_jobs)(delayed(score_track)(tid, corpus, path) for tid in tqdm(ids))
    results = {}
    for r in results_arr:
        results.update(r)
        
    return pd.DataFrame.from_dict(results, orient='index').sort_index(axis=1)

In [43]:
REF_PATH = '/home/bmcfee/data/eric_chords/references_v2/'

In [44]:
# MARL-mir analysis
marl_ids = pd.read_csv('/home/bmcfee/git/ace-lessons/data/chord_dataset/marl-chords.csv', squeeze=True, header=None)
marl_corp = 'MARL-Chords'

In [45]:
marl_cr2si = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_struct_aug/predictions-inv/')

marl_cr2s = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_struct_aug/predictions/')

marl_cr2 = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_aug/predictions/')

marl_cr1si = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/model_struct_aug/predictions-inv/')

marl_cr1s = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/model_struct_aug/predictions/')

marl_cr1 = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/model_aug/predictions/')

#marl_chordino = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/chordino/')

marl_dnn = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/ejh2015_dnn/predictions/')

marl_khmm = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/ejh2015_khmm/predictions/')

In [46]:
#marl_chordino.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_chordino_hier.json')
marl_dnn.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_dnn_hier.json')
marl_khmm.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_khmm_hier.json')

marl_cr1.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_cr1_hier.json')
marl_cr1s.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_cr1s_hier.json')
marl_cr1si.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_cr1si_hier.json')

marl_cr2.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_cr2_hier.json')
marl_cr2s.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_cr2s_hier.json')
marl_cr2si.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_cr2si_hier.json')


In [47]:
# Isophonics
iso_ids = pd.read_csv('/home/bmcfee/git/ace-lessons/data/chord_dataset/isophonics.csv', squeeze=True, header=None)
iso_corp = 'Isophonics'

In [48]:
iso_cr2si = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_struct_aug/predictions-inv/')

iso_cr2s = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_struct_aug/predictions/')

iso_cr2 = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_aug/predictions/')

iso_cr1si = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/model_struct_aug/predictions-inv/')

iso_cr1s = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/model_struct_aug/predictions/')

iso_cr1 = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/model_aug/predictions/')

#iso_chordino = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/chordino/')

iso_dnn = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/ejh2015_dnn/predictions/')

iso_khmm = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/ejh2015_khmm/predictions/')

In [49]:
#iso_chordino.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_chordino_hier.json')
iso_dnn.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_dnn_hier.json')
iso_khmm.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_khmm_hier.json')

iso_cr1.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_cr1_hier.json')
iso_cr1s.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_cr1s_hier.json')
iso_cr1si.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_cr1si_hier.json')

iso_cr2.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_cr2_hier.json')
iso_cr2s.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_cr2s_hier.json')
iso_cr2si.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_cr2si_hier.json')


In [50]:
# Billboard analysis
bb_ids = pd.read_csv('/home/bmcfee/git/ace-lessons/data/chord_dataset/billboard.csv', squeeze=True, header=None)
bb_corp = 'Billboard-Chords'

In [51]:
bb_cr2si = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_struct_aug/predictions-inv/')

bb_cr2s = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_struct_aug/predictions/')

bb_cr2 = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_aug/predictions/')

bb_cr1si = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/model_struct_aug/predictions-inv/')

bb_cr1s = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/model_struct_aug/predictions/')

bb_cr1 = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/model_aug/predictions/')

#bb_chordino = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/chordino/')

bb_dnn = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/ejh2015_dnn/predictions/')

bb_khmm = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/ejh2015_khmm/predictions/')

In [52]:
#bb_chordino.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_chordino_hier.json')
bb_dnn.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_dnn_hier.json')
bb_khmm.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_khmm_hier.json')

bb_cr1.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_cr1_hier.json')
bb_cr1s.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_cr1s_hier.json')
bb_cr1si.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_cr1si_hier.json')

bb_cr2.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_cr2_hier.json')
bb_cr2s.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_cr2s_hier.json')
bb_cr2si.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_cr2si_hier.json')


In [53]:
bb_khmm.describe()

,L-Measure,L-Precision,L-Recall,T-Measure full,T-Measure reduced,T-Precision full,T-Precision reduced,T-Recall full,T-Recall reduced
count,738.000000,738.000000,738.000000,738.000000,738.0,738.000000,738.0,738.000000,738.0
mean,0.653657,0.656786,0.656043,0.648538,0.0,0.583595,0.0,0.754649,0.0
std,0.150062,0.149431,0.152027,0.148036,0.0,0.172536,0.0,0.123593,0.0
min,0.010482,0.007551,0.017132,0.094292,0.0,0.122489,0.0,0.065365,0.0
25%,0.566694,0.564339,0.569133,0.571450,0.0,0.464312,0.0,0.724162,0.0
50%,0.669802,0.673162,0.666617,0.674685,0.0,0.603826,0.0,0.783590,0.0
75%,0.766886,0.773568,0.775492,0.754707,0.0,0.719957,0.0,0.828606,0.0
max,0.946258,0.955641,0.951384,0.919813,0.0,0.909408,0.0,0.935270,0.0


In [54]:
bb_dnn.describe()

,L-Measure,L-Precision,L-Recall,T-Measure full,T-Measure reduced,T-Precision full,T-Precision reduced,T-Recall full,T-Recall reduced
count,738.000000,738.000000,738.000000,738.000000,738.0,738.000000,738.0,738.000000,738.0
mean,0.652781,0.660091,0.651485,0.644530,0.0,0.568557,0.0,0.770084,0.0
std,0.165152,0.161665,0.169576,0.154272,0.0,0.174078,0.0,0.125363,0.0
min,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
25%,0.554141,0.558964,0.548233,0.565257,0.0,0.455665,0.0,0.736425,0.0
50%,0.669180,0.681866,0.669481,0.675833,0.0,0.585007,0.0,0.801192,0.0
75%,0.778982,0.783180,0.785465,0.757155,0.0,0.703153,0.0,0.843885,0.0
max,0.971134,0.967708,0.982509,0.919614,0.0,0.902510,0.0,0.937378,0.0


In [55]:
bb_cr2si.describe()

,L-Measure,L-Precision,L-Recall,T-Measure full,T-Measure reduced,T-Precision full,T-Precision reduced,T-Recall full,T-Recall reduced
count,738.000000,738.000000,738.000000,738.000000,738.0,738.000000,738.0,738.000000,738.0
mean,0.677357,0.677136,0.683786,0.662780,0.0,0.609517,0.0,0.746259,0.0
std,0.155820,0.155799,0.156487,0.119608,0.0,0.149486,0.0,0.100112,0.0
min,0.105973,0.057309,0.068388,0.134901,0.0,0.138033,0.0,0.118678,0.0
25%,0.577905,0.570678,0.580266,0.590138,0.0,0.507703,0.0,0.697324,0.0
50%,0.696937,0.694662,0.707670,0.670253,0.0,0.617781,0.0,0.762439,0.0
75%,0.802949,0.803356,0.808796,0.747612,0.0,0.722210,0.0,0.814520,0.0
max,0.970100,0.972578,0.987116,0.932070,0.0,0.935880,0.0,0.941823,0.0


In [56]:
bb_cr2s.describe()

,L-Measure,L-Precision,L-Recall,T-Measure full,T-Measure reduced,T-Precision full,T-Precision reduced,T-Recall full,T-Recall reduced
count,738.000000,738.000000,738.000000,738.000000,738.0,738.000000,738.0,738.000000,738.0
mean,0.676008,0.677525,0.680707,0.662780,0.0,0.609517,0.0,0.746259,0.0
std,0.156151,0.155839,0.157157,0.119608,0.0,0.149486,0.0,0.100112,0.0
min,0.106045,0.057352,0.068388,0.134901,0.0,0.138033,0.0,0.118678,0.0
25%,0.574454,0.570563,0.574803,0.590138,0.0,0.507703,0.0,0.697324,0.0
50%,0.694601,0.695757,0.702266,0.670253,0.0,0.617781,0.0,0.762439,0.0
75%,0.801675,0.802457,0.805958,0.747612,0.0,0.722210,0.0,0.814520,0.0
max,0.970100,0.972578,0.987116,0.932070,0.0,0.935880,0.0,0.941823,0.0
